<a href="https://colab.research.google.com/github/inhamjchoi/SafetyDataClass/blob/main/Ex06_1_Practice_1_ACC_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.signal import find_peaks
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def extract_features_from_window(data_x, data_y, data_z):
    """
    Extract features from a single window of X, Y, Z data

    Args:
        data_x, data_y, data_z: 1D arrays of window data

    Returns:
        dict: Dictionary containing features for this window
    """
    features = {}

    # Time domain features
    features['MeanX'] = np.mean(data_x)
    features['MeanY'] = np.mean(data_y)
    features['MeanZ'] = np.mean(data_z)

    features['SkewX'] = stats.skew(data_x)
    features['SkewY'] = stats.skew(data_y)
    features['SkewZ'] = stats.skew(data_z)

    features['MaxX'] = np.max(data_x)
    features['MaxY'] = np.max(data_y)
    features['MaxZ'] = np.max(data_z)

    features['MinX'] = np.min(data_x)
    features['MinY'] = np.min(data_y)
    features['MinZ'] = np.min(data_z)

    features['RangeX'] = np.max(data_x) - np.min(data_x)
    features['RangeY'] = np.max(data_y) - np.min(data_y)
    features['RangeZ'] = np.max(data_z) - np.min(data_z)

    features['StdX'] = np.std(data_x, ddof=1)
    features['StdY'] = np.std(data_y, ddof=1)
    features['StdZ'] = np.std(data_z, ddof=1)

    features['KurtosisX'] = stats.kurtosis(data_x, fisher=False)
    features['KurtosisY'] = stats.kurtosis(data_y, fisher=False)
    features['KurtosisZ'] = stats.kurtosis(data_z, fisher=False)

    # Average peak distance
    peaks_x, _ = find_peaks(data_x)
    if len(peaks_x) > 1:
        features['Avg_peak_distX'] = np.mean(np.diff(peaks_x))
    else:
        features['Avg_peak_distX'] = 0

    peaks_y, _ = find_peaks(data_y)
    if len(peaks_y) > 1:
        features['Avg_peak_distY'] = np.mean(np.diff(peaks_y))
    else:
        features['Avg_peak_distY'] = 0

    peaks_z, _ = find_peaks(data_z)
    if len(peaks_z) > 1:
        features['Avg_peak_distZ'] = np.mean(np.diff(peaks_z))
    else:
        features['Avg_peak_distZ'] = 0

    # Correlations
    features['CorrXY'] = np.corrcoef(data_x, data_y)[0, 1]
    features['CorrYZ'] = np.corrcoef(data_y, data_z)[0, 1]
    features['CorrZX'] = np.corrcoef(data_z, data_x)[0, 1]

    # Frequency domain features
    # Sampling frequency for ACC1
    fs = 32  # Hz
    window_length = len(data_x)

    # Energy calculation
    def compute_energy(signal):
        # Remove DC component
        new_signal = signal - np.mean(signal)
        # Zero-pad to next power of 2
        n = 2 ** int(np.ceil(np.log2(window_length)))
        # Compute FFT
        dft = np.fft.fft(new_signal, n)
        # Power spectrum (one-sided)
        power = np.abs(dft[:n//2])**2 / n
        return np.sum(power)

    features['EnergyX'] = compute_energy(data_x)
    features['EnergyY'] = compute_energy(data_y)
    features['EnergyZ'] = compute_energy(data_z)

    # Entropy calculation
    def compute_entropy(signal):
        # Zero-pad to next power of 2
        n = 2 ** int(np.ceil(np.log2(window_length)))
        # Compute FFT
        dft = np.fft.fft(signal, n)
        # Power spectrum (one-sided)
        power = np.abs(dft[:n//2])**2 / n
        # Normalize
        temp = power - np.min(power)
        if np.max(temp) - np.min(temp) == 0:
            return 0
        norm_pow = 1e-12 + temp / (np.max(temp) - np.min(temp))
        # Entropy
        entropy = -np.sum(norm_pow * np.log2(norm_pow + 1e-12))
        return entropy

    features['EntropyX'] = compute_entropy(data_x)
    features['EntropyY'] = compute_entropy(data_y)
    features['EntropyZ'] = compute_entropy(data_z)

    return features

In [ ]:
if __name__ == "__main__":

    print("=== ACC1.csv 1-Second Window Feature Extraction ===\n")

    # Read accelerometer data from ACC1.csv
    # - The first row contains the recording start time (not needed for analysis)
    # - The second row contains the sampling frequency in Hz (32 Hz)
    # - We skip both using `skiprows=2`
    # - The actual sensor data starts from the 3rd row onward (acceleration in X, Y, Z)
    df = pd.read_csv('ACC1.csv', header=None, skiprows=2)

    # Rename columns to indicate axis of acceleration
    df.columns = ['X', 'Y', 'Z']

    # Preview the first few rows
    print("Raw data preview:")
    print(df.head())
    print(f"Data shape: {df.shape}")

    # Extract raw data for each axis
    rawdata_x = df['X'].values
    rawdata_y = df['Y'].values
    rawdata_z = df['Z'].values

    print(f"\nData lengths:")
    print(f"Total data points: {len(rawdata_x)}")

    print(f"\nData ranges:")
    print(f"X: {np.min(rawdata_x):.3f} to {np.max(rawdata_x):.3f}")
    print(f"Y: {np.min(rawdata_y):.3f} to {np.max(rawdata_y):.3f}")
    print(f"Z: {np.min(rawdata_z):.3f} to {np.max(rawdata_z):.3f}")

    print(f"\nBasic statistics:")
    print(f"Mean - X: {np.mean(rawdata_x):.3f}, Y: {np.mean(rawdata_y):.3f}, Z: {np.mean(rawdata_z):.3f}")
    print(f"Std  - X: {np.std(rawdata_x):.3f}, Y: {np.std(rawdata_y):.3f}, Z: {np.std(rawdata_z):.3f}")

    # Window parameters
    sampling_rate = 32  # Hz
    window_duration = 1  # seconds
    window_size = sampling_rate * window_duration  # 32 * 1 = 32 data points

    print(f"\nWindow parameters:")
    print(f"Sampling rate: {sampling_rate} Hz")
    print(f"Window duration: {window_duration} seconds")
    print(f"Window size: {window_size} data points")

    # Calculate number of windows (no overlap)
    total_data_points = len(rawdata_x)
    num_windows = total_data_points // window_size

    print(f"Total data points: {total_data_points}")
    print(f"Number of 1-second windows: {num_windows}")
    print(f"Data points used: {num_windows * window_size}")
    print(f"Data points remaining: {total_data_points - (num_windows * window_size)}")

    # Create 1-second windows (no overlap)
    print(f"\nCreating 1-second windows...")

    windows_x = []
    windows_y = []
    windows_z = []

    for i in range(num_windows):
        start_idx = i * window_size
        end_idx = start_idx + window_size

        # Extract window data
        window_x = rawdata_x[start_idx:end_idx]
        window_y = rawdata_y[start_idx:end_idx]
        window_z = rawdata_z[start_idx:end_idx]

        windows_x.append(window_x)
        windows_y.append(window_y)
        windows_z.append(window_z)

    print(f"Created {len(windows_x)} windows of {window_size} points each")

    # Initialize results storage
    results = []

    print(f"\nExtracting features from each 1-second window...")

    # Process each 1-second window
    for i in range(num_windows):
        if (i + 1) % 10 == 0:
            print(f"Processing window {i+1}/{num_windows}...")

        # Get window data
        data_x = windows_x[i]
        data_y = windows_y[i]
        data_z = windows_z[i]

        # Extract features for this window
        features = extract_features_from_window(data_x, data_y, data_z)

        # Add window information
        features['Window_ID'] = i + 1
        features['Time_Start'] = i * window_duration  # Start time in seconds
        features['Time_End'] = (i + 1) * window_duration  # End time in seconds

        results.append(features)

    # Convert results to DataFrame
    feature_names = ['MeanX', 'MeanY', 'MeanZ', 'SkewX', 'SkewY', 'SkewZ',
                     'MaxX', 'MaxY', 'MaxZ', 'MinX', 'MinY', 'MinZ',
                     'RangeX', 'RangeY', 'RangeZ', 'StdX', 'StdY', 'StdZ',
                     'KurtosisX', 'KurtosisY', 'KurtosisZ',
                     'Avg_peak_distX', 'Avg_peak_distY', 'Avg_peak_distZ',
                     'CorrXY', 'CorrYZ', 'CorrZX',
                     'EnergyX', 'EnergyY', 'EnergyZ',
                     'EntropyX', 'EntropyY', 'EntropyZ',
                     'Window_ID', 'Time_Start', 'Time_End']

    all_result = pd.DataFrame(results, columns=feature_names)

    print(f"\n=== Feature Extraction Complete ===")
    print(f"Total windows processed: {len(all_result)}")
    print(f"Features per window: 33 + 3 metadata columns")

    # Display sample results
    print(f"\nSample of extracted features:")
    display_cols = ['Window_ID', 'Time_Start', 'Time_End', 'MeanX', 'MeanY', 'MeanZ', 'StdX', 'StdY', 'StdZ']
    print(all_result[display_cols].head().round(4))

    # Display some statistics
    print(f"\nFeature statistics (first few features):")
    stats_cols = ['MeanX', 'MeanY', 'MeanZ', 'StdX', 'StdY', 'StdZ']
    print(all_result[stats_cols].describe().round(4))

    # Save results to CSV
    output_file = 'ACC1_1second_windows_features.csv'
    all_result.to_csv(output_file, index=False)
    print(f"\nResults saved to: {output_file}")

    # Additional summary
    print(f"\n=== Summary ===")
    print(f"Original data points: {total_data_points}")
    print(f"Sampling rate: {sampling_rate} Hz")
    print(f"Window duration: {window_duration} seconds")
    print(f"Window size: {window_size} data points")
    print(f"Total windows: {num_windows}")
    print(f"Total recording time: {total_data_points/sampling_rate:.1f} seconds")
    print(f"Analyzed time: {num_windows * window_duration} seconds")
    print(f"Features per window: 33")
    print(f"Output file: {output_file}")

    print("\n=== Processing Complete ===")

=== ACC1.csv 1-Second Window Feature Extraction ===

Raw data preview:
   X   Y   Z
0 -2  59  29
1  4  57  29
2  2  59  28
3  2  60  29
4  2  60  31
Data shape: (177, 3)

Data lengths:
Total data points: 177

Data ranges:
X: -32.000 to 126.000
Y: -7.000 to 93.000
Z: -82.000 to 70.000

Basic statistics:
Mean - X: 17.458, Y: 51.582, Z: 15.537
Std  - X: 21.988, Y: 17.479, Z: 24.560

Window parameters:
Sampling rate: 32 Hz
Window duration: 1 seconds
Window size: 32 data points
Total data points: 177
Number of 1-second windows: 5
Data points used: 160
Data points remaining: 17

Creating 1-second windows...
Created 5 windows of 32 points each

Extracting features from each 1-second window...

=== Feature Extraction Complete ===
Total windows processed: 5
Features per window: 33 + 3 metadata columns

Sample of extracted features:
   Window_ID  Time_Start  Time_End    MeanX    MeanY    MeanZ     StdX  \
0          1           0         1   5.9062  59.0625  27.0625   6.1506   
1          2     